In [1]:
import os
import sys
import json
from os.path import join, basename, dirname

import numpy as np
import pandas as pd

import time
from tqdm import tqdm

from feature_analyzer.index.agent import IndexAgent
from feature_analyzer.data_tools.embedding_container import EmbeddingContainer
from feature_analyzer.utils.template_parser import InferenceResult

In [2]:
data_dir = '/home/kv_zhao/nist-e2e/feature-analyzer/examples/featobj_mergeV1_D40kv2_RMG/'
data_dir = '/Users/kv/workspace/feature-analyzer/features/D40kRM'

In [3]:
#RMG = InferenceResult('/home/kv_zhao/nist-e2e/outcomes/MERGE_V1/D40kv2_RMG_iv1_pv1/')
RMG = InferenceResult('/Users/kv/workspace/nist-e2e/outcomes/MERGE_V0/D40kv2_Rms1m_clean/')

Load from /Users/kv/workspace/nist-e2e/outcomes/MERGE_V0/D40kv2_Rms1m_clean/
FP: 23466, FN: 1316, NF: 16, WLM:1203


In [4]:
container = EmbeddingContainer()
container.load(data_dir)

Container:embedding_container created
Load embedding container from feat_obj format
/Users/kv/workspace/feature-analyzer/features/D40kRM/filename_strings.npy is loaded
/Users/kv/workspace/feature-analyzer/features/D40kRM/label_ids.npy is loaded
/Users/kv/workspace/feature-analyzer/features/D40kRM/instance_ids.npy is loaded
/Users/kv/workspace/feature-analyzer/features/D40kRM/embeddings.npy is loaded
/Users/kv/workspace/feature-analyzer/features/D40kRM/label_names.npy is loaded
/Users/kv/workspace/feature-analyzer/features/D40kRM/landmarks.npy is loaded
/Users/kv/workspace/feature-analyzer/features/D40kRM/probabilities.npy is loaded
container size: 10000 -> 120429
embedding size: 0 -> 1024
probability size: 0 -> 1
landmark size: 0 -> 10
Reset embedding_container
Index Table Created
Container initialized.


In [5]:
instance_ids = container.instance_ids
all_embeddings = container.get_embedding_by_instance_ids(instance_ids)
agent = IndexAgent('HNSW', instance_ids, all_embeddings, distance_measure='ip')

HNSW Index Agent is initialized with 120429 features


In [6]:
print(container)

=============== embedding_container ===============
embeddings: (120429, 1024)
probabilities: (120429, 1)
landamrks: (120429, 10)
internals: instance_ids, label_ids, label_names, filename_strings
attributes: source



In [ ]:
start = time.time()
label_ids = list(set(container.label_ids))
num_identity = len(label_ids)

for label_id in label_ids[2:3]:
    same_class_inst_ids = container.get_instance_ids_by_label_ids(label_id)
    same_class_embeddings = container.get_embedding_by_instance_ids(same_class_inst_ids)
    class_center_embedding = np.mean(same_class_embeddings, axis=0)
    class_center_fluct = np.mean(np.std(same_class_embeddings, axis=0))
    
    # instance
    num_inst_same_class = len(same_class_inst_ids)
    retrieved_indexes, similarities = agent.search(
        same_class_embeddings, top_k = 2 * num_inst_same_class, is_similarity=True)
    retrieved_label_ids = container.get_label_by_instance_ids(retrieved_indexes)
    hits = retrieved_label_ids == np.asarray([label_id])
    
    # top k instance
    topk_hits = hits[:, :num_inst_same_class]
    #np.isin(retrieved_indexes[:, :num_inst_same_class], same_class_inst_ids)
    topk_hit_counts = np.sum(topk_hits, axis=1)
    topk_miss_counts = np.sum(~topk_hits, axis=1)
    topk_purities = topk_hit_counts / num_inst_same_class
    topk_same_class_purity = np.mean(topk_purities)
    
    print(topk_miss_counts, topk_purities, topk_same_class_purity)
    
    # center
    center_retrieved_indexes, center_similarities = agent.search(
        class_center_embedding, top_k = 2 * num_inst_same_class, is_similarity=True)
    center_retrieved_label_ids = container.get_label_by_instance_ids(center_retrieved_indexes)
    center_hits = center_retrieved_label_ids == np.asarray([label_id])
    
    # top k center
    topk_center_hits = center_hits[:, :num_inst_same_class]
    topk_center_hit_counts = np.sum(topk_center_hits, axis=1)
    topk_center_purities = topk_center_hit_counts / num_inst_same_class
    topk_center_same_class_purity = np.mean(topk_center_purities)
    
    
    # top 2k instance
    assert hits.shape == retrieved_label_ids.shape
    for row, (hit_arr, hit_label_arr) in enumerate(zip(hits, retrieved_label_ids)):
        # for each arr, must have negative and positive (self)
        #hit_label_arr = retrieved_label_ids[row]
        first_neg_id = np.argmax(~hit_arr)
        prev_pos_id = first_neg_id - 1
        last_pos_id = np.where(hit_arr)[-1][-1]
        last_pos_sim = similarities[row, last_pos_id]
        first_neg_sim = similarities[row, first_neg_id]
        margin = first_neg_sim - last_pos_sim
        print(first_neg_id, last_pos_id, hit_arr, hit_label_arr, margin)
        
    #last_positive_ids = np.asarray(last_positive_ids)
    #first_negative_ids = np.asarray(first_negative_ids)
    

    # if first_neg > last_pos (purity == 1.0) => compute margin
    # otherwise, count how many different classes within.
    #print('hits', hit_label_ids)
    #print('positive ids', positive_ids)
    #print('negative ids', negative_ids)

    mean_purity += same_class_purity
    
    counter += 1
    #break
    
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))